In [1]:
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from IPython import display
from unityagents import UnityEnvironment

from utils import create_next_id_folder
from ddpg_agent_multi import Agent

In [2]:
# Create simulation environment
env = UnityEnvironment(file_name='Reacher_Windows_x86_64_20/Reacher.exe')
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [4]:
states.shape

(20, 33)

In [5]:
len(env_info.rewards)

20

In [6]:
# Initialize Feed-forward DNNs for Actor and Critic models. 
agent = Agent(state_size=state_size, action_size=action_size, random_seed=0, num_agents=num_agents)

env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
steps = 0
while True:
    steps += 1 
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.1169999973848462


In [7]:
scores

array([0.        , 0.47999999, 0.        , 0.        , 0.07      ,
       0.        , 0.        , 0.        , 0.24999999, 0.27999999,
       0.12      , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05      , 0.70999998, 0.37999999, 0.        ])

In [8]:
scores / steps

array([0.00000000e+00, 4.79520469e-04, 0.00000000e+00, 0.00000000e+00,
       6.99300684e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.49750244e-04, 2.79720273e-04, 1.19880117e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.99500488e-05, 7.09290693e-04, 3.79620371e-04, 0.00000000e+00])

In [9]:
from shared_agent import SharedAgent

def ddpg(env, 
        state_size, 
        action_size, 
        random_seed,
        num_agents,
        n_episodes=2000, 
        max_t=1000, 
        eps_start=1.0, 
        eps_end=0.01, 
        eps_decay=0.995, 
        actor_model_save_path='actor.pth',
        critic_model_save_path='critic.pth',
        BUFFER_SIZE = int(1e4),  # replay buffer size
        BATCH_SIZE = 128,        # minibatch size
        GAMMA = 0.99,            # discount factor
        TAU = 1e-3,              # for soft update of target parameters
        LR_ACTOR = 1e-4,         # learning rate of the actor 
        LR_CRITIC = 1e-3,        # learning rate of the critic
        WEIGHT_DECAY = 0,        # L2 weight decay
        learn_every=20, 
        learn_iterations=10,
        ):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores_agents = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    agent = SharedAgent(state_size=state_size, action_size=action_size, 
                    num_agents=num_agents,
                    random_seed=random_seed,
                    buffer_size=BUFFER_SIZE,                  
                    batch_size=BATCH_SIZE,
                    gamma=GAMMA,
                    tau=TAU,
                    actor_lr=LR_ACTOR,
                    critic_lr=LR_CRITIC,
                    weight_decay=WEIGHT_DECAY,
                    learn_every=learn_every,
                    learn_iterations=learn_iterations,
                  )

    for i_episode in range(1, n_episodes+1):

        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations                  # get the current state
        scores = np.zeros(num_agents)
        for t in range(max_t):
            actions = agent.act(states)                        # select an action (for each agent)
            actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent.step(states, actions, rewards, next_states, dones)
            scores += env_info.rewards
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        scores_window.append(scores)                      # save most recent score
        scores_agents.append(scores)                      # save most recent score

        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))

    torch.save(agent.actor_local.state_dict(), actor_model_save_path)
    torch.save(agent.critic_local.state_dict(), critic_model_save_path)
    return scores_agents

In [11]:
import os
import json
random_seed = 0

base_path = 'ddpg_trials'
combinations = [{
        "n_episodes": 2000,
        "max_t": 1000,
        "eps_start": 1.0,
        "eps_end": 0.01,
        "eps_decay": 0.995,
        "BUFFER_SIZE": int(1e4),
        "BATCH_SIZE": 128,
        "GAMMA": 0.99,
        "TAU": 1e-3,
        "LR_ACTOR": 1e-4,
        "LR_CRITIC": 1e-3,
        "WEIGHT_DECAY": 0,
    },
 ]

for properties in combinations:
    new_path = create_next_id_folder(base_path)
    scores = ddpg(env, 
        state_size, 
        action_size, 
        random_seed,
        num_agents=num_agents,
        n_episodes=properties["n_episodes"],
        max_t=properties["max_t"],
        eps_start=properties["eps_start"],
        eps_end=properties["eps_end"],
        eps_decay=properties["eps_decay"],
        actor_model_save_path=os.path.join(new_path, 'actor.pth'),
        critic_model_save_path=os.path.join(new_path, 'critic.pth'),
        BUFFER_SIZE=properties["BUFFER_SIZE"],
        BATCH_SIZE=properties["BATCH_SIZE"],
        GAMMA=properties["GAMMA"],
        TAU=properties["TAU"],
        LR_ACTOR=properties["LR_ACTOR"],
        LR_CRITIC=properties["LR_CRITIC"],
        WEIGHT_DECAY=properties["WEIGHT_DECAY"],
        )
    
    with open(os.path.join(new_path, 'parameters.json'), 'w') as json_file:
        json.dump(properties, json_file,  indent=4)

    list_of_python_lists = []
    for arr in scores:
        list_of_python_lists.append(arr.tolist())

    with open(os.path.join(new_path, 'scores.json'), 'w') as f:
        json.dump(list_of_python_lists, f, indent=4) # indent makes it human-readable

    for i, score in enumerate(list_of_python_lists): 
        # plot the scores
        fig = plt.figure()
        ax = fig.add_subplot(111)
        plt.plot(np.arange(len(score)), score)
        plt.ylabel('Score')
        plt.xlabel('Episode #')
        plt.title(f"Agent {i+1}'s Scores")
        plt.show()

Successfully created directory: 'ddpg_trials\4'


ValueError: operands could not be broadcast together with shapes (20,4) (20,) 